In [1]:
import numpy as np

In [93]:
class Symbol:
    """Class for abstract symbols"""
    
    def __init__(self, name):
        self.name = name
        self.eval = None
    
    def __add__(self, other):
        pass

def poly_add(P, Q):
    """Addition of polynomials"""
    Poly = dict()
    for k in set(P.keys()).union(Q.keys()):
        P_val, Q_val = P.get(k), Q.get(k)
        Poly[k] = (0 if P_val is None else P_val) + (0 if Q_val is None else Q_val)
    return Poly
    
def poly_mul(P, Q):
    """Multiplication of polynomials"""
    Poly = dict()
    for a in P.keys():
        for b in Q.keys():
            if Poly.get(a + b) == None:
                Poly[a + b] = P[a] * Q[b]
            else:
                Poly[a + b] += P[a] * Q[b]
    return Poly
    
def scalar_mul(c, P):
    """Scalar multiplication"""
    return poly_mul({0 : c}, P)
    
def poly_diff(P):
    """Differentiation of polynomials"""
    Poly = dict()
    for k in P.keys():
        if k > 0:
            Poly[k - 1] = k * P[k]
    if len(Poly) == 0:
        Poly = {0 : 0}
    return Poly

def mul_picard_fuchs(A, B):
    """Multiplication of Picard-Fuchs type operators"""
    N = []
    for a in A:
        for b in B:
            N.append([a[0] @ b[0], a[1].mul(b[1])])
    return N

def diff_picard_fuchs(A):
    """Differentiation of a Picard-Fuchs type operator"""
    N = []
    for a in A:
        N.append([a[0], a[1].diff()])
    return N
        
class Rational:
    """Class for rational function"""
    
    def __init__(self, num=dict(), denom=dict()):
        self.num = num
        self.denom = denom
        
    def diff(self):
        """Differentiates a rational function via quotient rule"""
        P1 = poly_mul(poly_diff(self.num), self.denom)
        P2 = scalar_mul(-1, poly_mul(poly_diff(self.denom), self.num))
        
        num = poly_add(P1, P2)
        denom = poly_mul(self.denom, self.denom)
        return Rational(num, denom)
    
    def mul(self, other):
        """Multiplies rationals"""
        num = poly_mul(self.num, other.num)
        denom = poly_mul(self.denom, other.denom)
        return Rational(num, denom)
        
def construct_picard_fuchs(matrices, rationals, k):
    """Constructs the Picard-Fuchs operator for a given collection of matrices and rational functions via the iterative formula given in the notes"""
    A_z = [[-1 * mat, rat] for mat, rat in zip(matrices, rationals)]
    D = [A_z, [[np.eye(len(matrices[0])), Rational({0 : 1}, {0: 1})]]]
    
    for r in range(1, k):
        new_D = [mul_picard_fuchs(A_z, D[0])]
        for j in range(1, r + 1):
            # Constructs the new coefficient via the iterative formula
            O = D[j-1] + mul_picard_fuchs(A_z, D[j]) + diff_picard_fuchs(D[j])
            new_D.append(O)
        new_D.append([[np.eye(len(matrices[0])), Rational({0 : 1}, {0: 1})]])
        D = new_D
    return D

In [106]:
# Testing

matrices = [
    np.array([[1, 1], [1, 1]]),
]

rationals = [Rational({0 : 1}, {0 : 3, 1 : 2})]

In [107]:
D = construct_picard_fuchs(matrices, rationals, 5)